In [ ]:
import os
from glob import glob

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.patches import Rectangle
from PIL import Image

os.chdir("..")

from utils import indexed_from_arr, load_colormap

In [ ]:
# Load data
CROPPED_DIR = "data/panels_c_cropped"

df_ai = pd.read_excel(os.path.join(CROPPED_DIR, "../ai_coverage_c.xlsx"))
df_hu = pd.read_excel(os.path.join(CROPPED_DIR, "../human_coverage_c.xlsx"))

image_paths = sorted(glob(os.path.join(CROPPED_DIR, "*.jpg")))
label_paths = sorted(glob(os.path.join(CROPPED_DIR, "*.png")))
assert len(image_paths) == len(label_paths)
print(len(image_paths), "images found")

In [ ]:
# Split name to info
df_ai[["Location", "Panel", "Month"]] = df_ai["Name"].str.split("_", expand=True)
df_ai["Panel"] = df_ai["Panel"].astype(int)
df_ai["Month"] = df_ai["Month"].str.replace("m", "").astype(int)

# Clean up
df_ai = df_ai.drop(["Name", "Entropy", "Date"], axis=1)
df_ai["Others"] = 0.
df_ai = df_ai.set_index(["Location", "Panel", "Month"]).sort_index()
df_ai.head()

In [ ]:
# Clean up human data
df_hu = df_hu.drop("Date", axis=1)
df_hu = df_hu.set_index(["Location", "Panel", "Month"]).sort_index()
df_hu.head()

In [ ]:
def occupation_mask(series, overwrite=(0, 1, 2)):
    occupation = np.zeros_like(series[0])

    for s in series:
        ow_mask = np.isin(occupation, overwrite)
        occupation[ow_mask] = s[ow_mask]

    return occupation

classes, palette = load_colormap()
classes["Others"] = classes.pop("empty")

In [ ]:
# Calculate direct attachments and their distributions
for loc in ("Anchorage", "Port"):
    for i in range(1, 6):
        name = f"{loc}_{i}_"
        series = [np.asarray(Image.open(p)) for p in label_paths if name in p]

        dist = np.bincount(occupation_mask(series).ravel(), minlength=len(classes))
        dist = np.roll(dist / dist.sum(), -1) # empty to Others

        df_ai.loc[loc, i, len(series) + 1] = dist

df_ai = df_ai.sort_index()

In [ ]:
df_hu.to_parquet(os.path.join(CROPPED_DIR, "../human_time_series_c.parquet"))
df_ai.to_parquet(os.path.join(CROPPED_DIR, "../ai_time_series_c.parquet"))

Figure 4: Comparison of manual and automated analysis

In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(10, 4))

width = 0.3
months = 3
inds = np.arange(1, months + 1)
colors = np.asarray(palette).reshape((-1, 3)) / 255
loc_map = {"Anchorage": "Melbourne", "Port": "Port Canaveral"}

for i, ax in enumerate(axes.ravel()):
    loc = "Anchorage" if i < 5 else "Port"

    for df, w in ((df_ai, -width), (df_hu, width)):
        cover_prev = 0
        for c in list(classes.keys()):
            cover = df.loc[loc, i % 5 + 1][c] * 100
            color = colors[classes[c]]
            ax.bar(inds + w / 1.7, cover[:months], width, bottom=0 if np.sum(cover_prev) == 0
                else cover_prev[:months], color=color, label=c)
            if df is not df_hu:
                ax.bar(months + 1, cover[months + 1], width, bottom=0 if np.sum(cover_prev) == 0
                    else cover_prev[months + 1], color=color, label=c)
            
            cover_prev += cover

    ax.set_xticks((1, 2, 3, 4))
    ax.set_xticklabels((1, 2, 3, "$\\alpha$"))

    ax.set_ylim((0, 100))

    if i < axes.shape[1]:
        ax.set_title(f"Panel {i+1}", pad=10)

    if i >= axes.shape[1]:
        ax.set_xlabel("Months")
    
    if not i % 5:
        ax.set_ylabel("Coverage [%]")
        ax.annotate(f"{loc_map[loc]} Site", xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - 5, 0),
                xycoords=ax.yaxis.label, textcoords='offset points',
                size='large', ha='right', va='center', rotation=90)

handles, labels = ax.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(), loc="lower center", ncol=6,
    bbox_to_anchor=(0.5, -0.16), frameon=False)

fig.tight_layout()

plt.annotate("Bars: $\it{left}$ U-Net, $\it{right}$ human", 
    (0.014, 0.15), xycoords="figure fraction")

plt.savefig("results/figures/time series.svg", bbox_inches='tight')

Figure 5a: Determination of macrofoulers in direct coating contact from image time series during fouling progression

In [ ]:
panels = [Image.open(p) for p in image_paths if "Port_2" in p]

# Calculate direct attachement and its distribution
series = [Image.open(p) for p in label_paths if "Port_2" in p]
occupation = occupation_mask([np.asarray(p) for p in series])
series.append(indexed_from_arr(occupation, palette))

dist = np.bincount(occupation.ravel(), minlength=len(classes))
dist = np.roll(dist / dist.sum() * 100, -1)
df_ai.loc["Port", 2, 4] = dist

# Create image
fig, axes = plt.subplots(4, 4, figsize=(9, 9), 
    gridspec_kw={'height_ratios': [4.5, 4.5, 5, 5], "hspace": 0.05, "wspace": 0.1})

height = 0.2
colors = np.asarray(palette).reshape((-1, 3)) / 255
rect_size = (1020, 810, 320, 320) # x, y, side lengths
rect = lambda: Rectangle(rect_size[:2], *rect_size[2:], linewidth=1.5, 
    edgecolor='y', facecolor='none', alpha=1)

# Transform rect to PIL coords
box = np.asarray(rect_size)
box = np.concatenate((box[:2], box[:2] + box[2:]))

for i, ((ax1, ax2, ax3, ax4), im) in enumerate(zip(axes.T, series), 1):
    if i < len(series):
        panel_im = panels[i-1].resize(im.size, resample=Image.LANCZOS)
        ax1.imshow(panel_im)
        ax1.add_patch(rect())
        ax3.imshow(panel_im.crop(box))
    ax2.imshow(im)
    ax2.add_patch(rect())
    ax4.imshow(im.crop(box))

    ax4.set_title(f"Month {i}" if i != len(series) else "Direct Attachment", 
        y=-0.2)

    if i == 1:
        ax1.set_yticks([])
        ax1.xaxis.set_visible(False)
        for s in ax1.spines.values(): s.set_visible(False)
        ax1.set_ylabel("Panel image", size="large", labelpad=10)

        ax2.set_yticks([])
        ax2.xaxis.set_visible(False)
        for s in ax2.spines.values(): s.set_visible(False)
        ax2.set_ylabel("Segmentation", size="large", labelpad=10)

        ax3.set_yticks([])
        ax3.xaxis.set_visible(False)
        for s in ax3.spines.values(): s.set_visible(False)
        ax3.set_ylabel("Panel image\nclose-up", size="large", labelpad=10)

        ax4.set_yticks([])
        ax4.xaxis.set_visible(False)
        for s in ax4.spines.values(): s.set_visible(False)
        ax4.set_ylabel("Segmentation\nclose-up", size="large", labelpad=10)
    else:
        ax1.set_axis_off()
        ax2.set_axis_off()
        ax3.set_axis_off()
        ax4.set_axis_off()

fig.tight_layout(pad=0)
fig.savefig("results/figures/attachement series.svg", dpi=150, bbox_inches='tight')

Figure 5b: Determination of macrofoulers in direct coating contact from image time series during fouling progression

In [ ]:
def succession_model(series):
    series = np.asarray(series)
    assert series.ndim == 3

    # Layer model
    layers = np.zeros_like(series)
    layers[0] = series[0]

    # Pointer for current top values
    top = np.zeros_like(layers[0])
    i, j = np.indices(top.shape) # for fancy indexing
    
    # Collapse series to layers
    for t in range(1, len(series)):
        changed = series[t] != series[top, i, j]

        # Advance top pointer and update layer model
        top[changed] += 1
        layers[top[changed], i[changed], j[changed]] = series[t, changed]

    return layers

In [ ]:
series = [np.asarray(Image.open(p)) for p in label_paths if "Port_2" in p]
series = np.stack(series)
smodel = succession_model(series)
smodel.shape

In [ ]:
layers, counts = np.unique(series.reshape((3, -1)).T, axis=0, return_counts=True)

fig, ax = plt.subplots()
size = 0.5

for phase in range(2): # Correct removal of minorites
    for idx in range(1, len(series) + 1):
        foulers = np.unique(layers[:,:idx], axis=0)
        vals = np.asarray([counts[np.all(layers[:,:idx] == f, axis=-1)].sum() for f in foulers])
        vals = vals / vals.sum()

        # Remove minorities
        minor = vals < 0.01

        mask = ~(layers[:,:idx] == foulers[minor, None]).all(-1).any(0)
        layers = layers[mask]
        counts = counts[mask]

        vals = vals[~minor] #/ vals[~minor].sum()
        foulers = foulers[~minor]

        # Draw
        if phase:
            r = 2 - (len(series) - idx) * size
            pie = ax.pie(vals, radius=r, colors=colors[foulers[:,-1]], 
                normalize=False, autopct="%.0f", pctdistance=0.71 + (idx-1) * 0.09,
                textprops={"color":"white", "size":"large", "weight": "bold"},
                wedgeprops=dict(width=size, edgecolor='w', linewidth=1.5))

fig.savefig("results/figures/succession model.svg", dpi=150, bbox_inches='tight')

Figure 5c: Determination of macrofoulers in direct coating contact from image time series during fouling progression

In [ ]:
names, counts = np.unique(occupation, return_counts=True)
counts = counts / counts.sum()

names = names[counts >= 0.01]
counts = counts[counts >= 0.01]

names = [next(k for k,v in classes.items() if v==n) for n in names]
dict(zip(names, counts.round(2)))

In [ ]:
series = [np.asarray(Image.open(p).crop(box)) for p in label_paths if "Port_2" in p]
series = np.stack(series)
smodel = succession_model(series)
smodel.shape

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = plt.axes(projection='3d')
ax.view_init(elev=25)

colors = np.asarray(palette).reshape((-1, 3)) / 255
levels = np.arange(len(colors) + 1) - .5

X = np.arange(smodel.shape[1])
Y = np.arange(smodel.shape[2])
X, Y = np.meshgrid(X, Y)

kwargs = {"zdir":'z', "colors":colors, "antialiased":True, "levels":levels}

ax.contourf(X, Y, np.ma.masked_equal(series[0], 0).T, offset=0, **kwargs)
ax.contourf(X, Y, np.ma.masked_equal(smodel[1], 0).T, offset=1, **kwargs)
ax.contourf(X, Y, np.ma.masked_equal(smodel[2], 0).T, offset=2, **kwargs)

ax.set_zlim3d(0, 2)
ax.set_xticks([])
ax.set_yticks([])
ax.set_zticks([0, 1, 2])
ax.set_zticklabels([])

fig.savefig("results/figures/layer model.svg", dpi=150, bbox_inches='tight')